In [ ]:
import cv2
import shutil
import numpy as np
from pathlib import Path

# ============================================================
# CONFIG
# ============================================================

OUT_ROOT = Path("/media/ramsey/data1TB/temp/fall_data/data_6_office_stand")

DIRS = {
    "crop":   OUT_ROOT / "cropped_person",
    "detect": OUT_ROOT / "detected",
    "init":   OUT_ROOT / "initial_csv",
    "feat":   OUT_ROOT / "features",
}

CLASSES = ["stand", "fall"]

WIN_W, WIN_H = 1920, 1080
IMG_W, IMG_H = 224, 640

WINDOW_NAME = "Fall / Stand Label Correction"

# ============================================================
# LOAD SAMPLES (PAIRS)
# ============================================================

samples = []
for cls in CLASSES:
    for img_path in sorted((DIRS["detect"] / cls).glob("*.jpg")):
        samples.append((cls, img_path.stem))

if not samples:
    raise RuntimeError("No samples found")

idx = 0
cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)
cv2.resizeWindow(WINDOW_NAME, WIN_W, WIN_H)

# ============================================================
# FILE OPERATIONS
# ============================================================

def delete_pair(cls, name):
    for d in DIRS.values():
        for ext in [".jpg", ".csv"]:
            p = d / cls / f"{name}{ext}"
            if p.exists():
                p.unlink()

def move_pair(cls, name):
    new_cls = "fall" if cls == "stand" else "stand"
    for d in DIRS.values():
        for ext in [".jpg", ".csv"]:
            src = d / cls / f"{name}{ext}"
            dst = d / new_cls / f"{name}{ext}"
            if src.exists():
                shutil.move(src, dst)

# ============================================================
# UI DRAW
# ============================================================

def draw_ui(img, cls, name):
    canvas = np.zeros((WIN_H, WIN_W, 3), dtype=np.uint8)

    # resize detected image to fixed size
    img = cv2.resize(img, (IMG_W, IMG_H), interpolation=cv2.INTER_AREA)

    # center position
    x0 = (WIN_W - IMG_W) // 2
    y0 = (WIN_H - IMG_H) // 2

    canvas[y0:y0+IMG_H, x0:x0+IMG_W] = img

    # header
    color = (0, 255, 0) if cls == "stand" else (0, 0, 255)
    cv2.putText(
        canvas,
        f"CLASS: {cls.upper()}",
        (60, 70),
        cv2.FONT_HERSHEY_SIMPLEX,
        2.0,
        color,
        4
    )

    cv2.putText(
        canvas,
        f"FILE: {name}",
        (60, 130),
        cv2.FONT_HERSHEY_SIMPLEX,
        1.0,
        (255, 255, 255),
        2
    )

    # footer help
    help_text = "[P] KEEP    [I] INVERT    [D] DELETE    [ESC] EXIT"
    (tw, th), _ = cv2.getTextSize(help_text, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)
    cv2.putText(
        canvas,
        help_text,
        ((WIN_W - tw)//2, WIN_H - 40),
        cv2.FONT_HERSHEY_SIMPLEX,
        1.0,
        (255, 255, 0),
        2
    )

    return canvas

# ============================================================
# MAIN LOOP
# ============================================================

while idx < len(samples):

    cls, name = samples[idx]
    img_path = DIRS["detect"] / cls / f"{name}.jpg"

    img = cv2.imread(str(img_path))
    if img is None:
        idx += 1
        continue

    ui = draw_ui(img, cls, name)
    cv2.imshow(WINDOW_NAME, ui)

    key = cv2.waitKey(0) & 0xFF

    if key == ord('p'):            # keep
        idx += 1

    elif key == ord('d'):          # delete
        delete_pair(cls, name)
        idx += 1

    elif key == ord('i'):          # invert
        move_pair(cls, name)
        idx += 1

    elif key == 27:                # ESC
        break

cv2.destroyAllWindows()
print("✔ Correction finished")


RuntimeError: No samples found